In [335]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, Row, ArrayType
from pyspark.sql import SQLContext
from pyspark.sql.functions import from_json, explode, col
from pyspark.sql import functions as F
import requests
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
from dotenv import load_dotenv
from datetime import datetime
import datetime
import json


In [4]:
spark = SparkSession.builder\
.master('local')\
.appName('SportifyTracks')\
.getOrCreate()
sql_context = SQLContext(spark)

/usr/local/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [268]:
spark.stop()

In [269]:
sc = SparkContext(master="local", appName="spotify-etl")

In [ ]:
sc

<SparkContext master=local appName=spotify-etl>

In [340]:
def extract():
    df = spark.read.json('./tracks.json', multiLine=True)
    return df

In [343]:
df = extract()
df.printSchema()

root
 |-- cursors: struct (nullable = true)
 |    |-- after: string (nullable = true)
 |    |-- before: string (nullable = true)
 |-- href: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- context: struct (nullable = true)
 |    |    |    |-- external_urls: struct (nullable = true)
 |    |    |    |    |-- spotify: string (nullable = true)
 |    |    |    |-- href: string (nullable = true)
 |    |    |    |-- type: string (nullable = true)
 |    |    |    |-- uri: string (nullable = true)
 |    |    |-- played_at: string (nullable = true)
 |    |    |-- track: struct (nullable = true)
 |    |    |    |-- album: struct (nullable = true)
 |    |    |    |    |-- album_group: string (nullable = true)
 |    |    |    |    |-- album_type: string (nullable = true)
 |    |    |    |    |-- artists: array (nullable = true)
 |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |   